<a href="https://colab.research.google.com/github/tancse/Recommendation_engine/blob/main/Recommendation_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LIGHTGBM Model for Product Recomendation engine**

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
data = pd.read_csv("styles.csv", on_bad_lines='skip')
data.head(5)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


# **DATA PREPARATION**

In [ ]:
data1 = data[data['masterCategory']=="Apparel"]
data1.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011.0,Casual,Inkfruit Mens Chain Reaction T-shirt


In [ ]:
data1['subCategory'].unique()

array(['Topwear', 'Bottomwear', 'Innerwear', 'Saree', 'Dress',
       'Loungewear and Nightwear', 'Apparel Set', 'Socks'], dtype=object)

In [ ]:
list = ['Innerwear', 'Socks']
data2 = data1[data1['subCategory'] != 'Innerwear']
data2 = data2[data2['subCategory'] != 'Socks']
data2.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011.0,Casual,Inkfruit Mens Chain Reaction T-shirt


In [ ]:
data2['subCategory'].unique()

array(['Topwear', 'Bottomwear', 'Saree', 'Dress',
       'Loungewear and Nightwear', 'Apparel Set'], dtype=object)

In [ ]:
data2=data2.drop(columns='year')
data2

,id,gender,mcat,scat,name,color,season,usage,brand
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,Casual,Peter England Men Party Blue Jeans
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,Casual,Puma Men Grey T-shirt
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,Casual,Inkfruit Mens Chain Reaction T-shirt
...,...,...,...,...,...,...,...,...,...
44414,30614,Men,Apparel,Topwear,Tshirts,Black,Summer,Sports,Nike Men Striped Black Jersey
44415,13496,Men,Apparel,Topwear,Tshirts,Blue,Fall,Casual,Chimp Men Teja Main Hoon Blue Tshirts
44417,12544,Women,Apparel,Topwear,Tshirts,Peach,Fall,Casual,Tantra Women Printed Peach T-shirt
44418,42234,Women,Apparel,Topwear,Tops,Blue,Summer,Casual,Sepia Women Blue Printed Top


In [ ]:
data2=data2.rename(columns={'articleType':'name', 'baseColour': 'color', 'productDisplayName': 'brand', 'masterCategory': 'mcat', 'subCategory':'scat'})
data2

,id,gender,mcat,scat,name,color,season,usage,brand
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,Casual,Peter England Men Party Blue Jeans
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,Casual,Puma Men Grey T-shirt
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,Casual,Inkfruit Mens Chain Reaction T-shirt
...,...,...,...,...,...,...,...,...,...
44414,30614,Men,Apparel,Topwear,Tshirts,Black,Summer,Sports,Nike Men Striped Black Jersey
44415,13496,Men,Apparel,Topwear,Tshirts,Blue,Fall,Casual,Chimp Men Teja Main Hoon Blue Tshirts
44417,12544,Women,Apparel,Topwear,Tshirts,Peach,Fall,Casual,Tantra Women Printed Peach T-shirt
44418,42234,Women,Apparel,Topwear,Tops,Blue,Summer,Casual,Sepia Women Blue Printed Top


In [ ]:
data2.to_csv('style.csv', index=False)

# **DATA PRE-PROCESSING**

In [ ]:
df = pd.read_csv('style.csv')
df.head()

,id,gender,mcat,scat,name,color,season,usage,brand
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,Casual,Peter England Men Party Blue Jeans
2,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,Casual,Manchester United Men Solid Black Track Pants
3,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,Casual,Puma Men Grey T-shirt
4,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,Casual,Inkfruit Mens Chain Reaction T-shirt


In [ ]:
df = df.iloc[:,[0,4,8,2,3,7,1,5,6]]
df.head()

,id,name,brand,mcat,scat,usage,gender,color,season
0,15970,Shirts,Turtle Check Men Navy Blue Shirt,Apparel,Topwear,Casual,Men,Navy Blue,Fall
1,39386,Jeans,Peter England Men Party Blue Jeans,Apparel,Bottomwear,Casual,Men,Blue,Summer
2,21379,Track Pants,Manchester United Men Solid Black Track Pants,Apparel,Bottomwear,Casual,Men,Black,Fall
3,53759,Tshirts,Puma Men Grey T-shirt,Apparel,Topwear,Casual,Men,Grey,Summer
4,1855,Tshirts,Inkfruit Mens Chain Reaction T-shirt,Apparel,Topwear,Casual,Men,Grey,Summer


In [ ]:
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.lower()

df

,id,name,brand,mcat,scat,usage,gender,color,season
0,15970,shirts,turtle check men navy blue shirt,apparel,topwear,casual,men,navy blue,fall
1,39386,jeans,peter england men party blue jeans,apparel,bottomwear,casual,men,blue,summer
2,21379,track pants,manchester united men solid black track pants,apparel,bottomwear,casual,men,black,fall
3,53759,tshirts,puma men grey t-shirt,apparel,topwear,casual,men,grey,summer
4,1855,tshirts,inkfruit mens chain reaction t-shirt,apparel,topwear,casual,men,grey,summer
...,...,...,...,...,...,...,...,...,...
19572,30614,tshirts,nike men striped black jersey,apparel,topwear,sports,men,black,summer
19573,13496,tshirts,chimp men teja main hoon blue tshirts,apparel,topwear,casual,men,blue,fall
19574,12544,tshirts,tantra women printed peach t-shirt,apparel,topwear,casual,women,peach,fall
19575,42234,tops,sepia women blue printed top,apparel,topwear,casual,women,blue,summer


In [ ]:
df['name'] = df['name'].astype(str)+' '+df['brand'].astype(str)+' '+df['mcat'].astype(str)+' '+df['scat'].astype(str)+' '+df['usage'].astype(str)+' '+df['gender'].astype(str)+' '+df['color'].astype(str)+' '+df['season'].astype(str)
df

,id,name,brand,mcat,scat,usage,gender,color,season
0,15970,shirts turtle check men navy blue shirt appare...,turtle check men navy blue shirt,apparel,topwear,casual,men,navy blue,fall
1,39386,jeans peter england men party blue jeans appar...,peter england men party blue jeans,apparel,bottomwear,casual,men,blue,summer
2,21379,track pants manchester united men solid black ...,manchester united men solid black track pants,apparel,bottomwear,casual,men,black,fall
3,53759,tshirts puma men grey t-shirt apparel topwear ...,puma men grey t-shirt,apparel,topwear,casual,men,grey,summer
4,1855,tshirts inkfruit mens chain reaction t-shirt a...,inkfruit mens chain reaction t-shirt,apparel,topwear,casual,men,grey,summer
...,...,...,...,...,...,...,...,...,...
19572,30614,tshirts nike men striped black jersey apparel ...,nike men striped black jersey,apparel,topwear,sports,men,black,summer
19573,13496,tshirts chimp men teja main hoon blue tshirts ...,chimp men teja main hoon blue tshirts,apparel,topwear,casual,men,blue,fall
19574,12544,tshirts tantra women printed peach t-shirt app...,tantra women printed peach t-shirt,apparel,topwear,casual,women,peach,fall
19575,42234,tops sepia women blue printed top apparel topw...,sepia women blue printed top,apparel,topwear,casual,women,blue,summer


In [ ]:
df.iloc[0,1]

'shirts turtle check men navy blue shirt apparel topwear casual men navy blue fall'

In [ ]:
df = df.drop(columns=['brand', 'mcat', 'scat', 'usage', 'gender', 'color', 'season'])
df

,id,name
0,15970,shirts turtle check men navy blue shirt appare...
1,39386,jeans peter england men party blue jeans appar...
2,21379,track pants manchester united men solid black ...
3,53759,tshirts puma men grey t-shirt apparel topwear ...
4,1855,tshirts inkfruit mens chain reaction t-shirt a...
...,...,...
19572,30614,tshirts nike men striped black jersey apparel ...
19573,13496,tshirts chimp men teja main hoon blue tshirts ...
19574,12544,tshirts tantra women printed peach t-shirt app...
19575,42234,tops sepia women blue printed top apparel topw...


In [ ]:
df = df.rename(columns={'name':'product'})
df

,id,product
0,15970,shirts turtle check men navy blue shirt appare...
1,39386,jeans peter england men party blue jeans appar...
2,21379,track pants manchester united men solid black ...
3,53759,tshirts puma men grey t-shirt apparel topwear ...
4,1855,tshirts inkfruit mens chain reaction t-shirt a...
...,...,...
19572,30614,tshirts nike men striped black jersey apparel ...
19573,13496,tshirts chimp men teja main hoon blue tshirts ...
19574,12544,tshirts tantra women printed peach t-shirt app...
19575,42234,tops sepia women blue printed top apparel topw...


In [ ]:
import re
import nltk
import string
from nltk.corpus import stopwords

# Download NLTK stopwords once
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def clean_text(text):

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Tokenize and remove stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]

    # Rejoin tokens
    return ' '.join(tokens)

In [ ]:
def remove_tags(text):
  text = re.sub(r'<.*?>', '', text)
  return text

In [ ]:
def remove_punctuation(text):
  text = text.translate(str.maketrans('', '', string.punctuation))
  return text

In [ ]:
def remove_numbers(text):
  text = re.sub(r'\d+', '', text)
  return text

In [ ]:
def tokenize(text):
  tokens = text.split()
  tokens = [word for word in tokens if word not in stop_words]
  return ' '.join(tokens)

In [ ]:
text = df.iloc[3,1]
clean_text(text)

'tshirts puma men grey tshirt apparel topwear casual men grey summer'

In [ ]:
for col in df['product']:
    df['product'] = df['product'].apply(remove_tags)

df

,id,product
0,15970,shirts turtle check men navy blue shirt appare...
1,39386,jeans peter england men party blue jeans appar...
2,21379,track pants manchester united men solid black ...
3,53759,tshirts puma men grey tshirt apparel topwear c...
4,1855,tshirts inkfruit mens chain reaction tshirt ap...
...,...,...
19572,30614,tshirts nike men striped black jersey apparel ...
19573,13496,tshirts chimp men teja main hoon blue tshirts ...
19574,12544,tshirts tantra women printed peach tshirt appa...
19575,42234,tops sepia women blue printed top apparel topw...


In [ ]:
for col in df['product']:
    df['product'] = df['product'].apply(remove_punctuation)

df

,id,product
0,15970,shirts turtle check men navy blue shirt appare...
1,39386,jeans peter england men party blue jeans appar...
2,21379,track pants manchester united men solid black ...
3,53759,tshirts puma men grey tshirt apparel topwear c...
4,1855,tshirts inkfruit mens chain reaction tshirt ap...
...,...,...
19572,30614,tshirts nike men striped black jersey apparel ...
19573,13496,tshirts chimp men teja main hoon blue tshirts ...
19574,12544,tshirts tantra women printed peach tshirt appa...
19575,42234,tops sepia women blue printed top apparel topw...


In [ ]:
for col in df['product']:
    df['product'] = df['product'].apply(remove_numbers)

df

,id,product
0,15970,shirts turtle check men navy blue shirt appare...
1,39386,jeans peter england men party blue jeans appar...
2,21379,track pants manchester united men solid black ...
3,53759,tshirts puma men grey tshirt apparel topwear c...
4,1855,tshirts inkfruit mens chain reaction tshirt ap...
...,...,...
19572,30614,tshirts nike men striped black jersey apparel ...
19573,13496,tshirts chimp men teja main hoon blue tshirts ...
19574,12544,tshirts tantra women printed peach tshirt appa...
19575,42234,tops sepia women blue printed top apparel topw...


In [ ]:
for col in df['product']:
    df['product'] = df['product'].apply(tokenize)

df

,id,product
0,15970,shirts turtle check men navy blue shirt appare...
1,39386,jeans peter england men party blue jeans appar...
2,21379,track pants manchester united men solid black ...
3,53759,tshirts puma men grey tshirt apparel topwear c...
4,1855,tshirts inkfruit mens chain reaction tshirt ap...
...,...,...
19572,30614,tshirts nike men striped black jersey apparel ...
19573,13496,tshirts chimp men teja main hoon blue tshirts ...
19574,12544,tshirts tantra women printed peach tshirt appa...
19575,42234,tops sepia women blue printed top apparel topw...


In [ ]:
df.to_csv('tokenized_style.csv', index=False)

# **SIMPLE RECOMMENDATION MODEL**

In [ ]:
df = pd.read_csv('tokenized_style.csv')
df.head()

,id,product
0,15970,shirts turtle check men navy blue shirt appare...
1,39386,jeans peter england men party blue jeans appar...
2,21379,track pants manchester united men solid black ...
3,53759,tshirts puma men grey tshirt apparel topwear c...
4,1855,tshirts inkfruit mens chain reaction tshirt ap...


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:

# Load your processed data
# Assume a DataFrame with columns: 'product_id' and 'product_text'
df = pd.read_csv("tokenized_style.csv")

In [ ]:
# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = vectorizer.fit_transform(df['product'])



In [ ]:
# Function to get relevant product IDs for a query
def search_products(query, top_k=10):
    query_vec = vectorizer.transform([query])
    similarity = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = similarity.argsort()[::-1][:top_k]
    return df.iloc[top_indices][['id', 'product']]


In [ ]:
# Example usage
query = "red coloured tops"
results = search_products(query)
print(results)

          id                                            product
12119  11034  tops jealous women check red tops apparel topw...
12292  11033  tops jealous women check red tops apparel topw...
14999  52118  tops myntra women multi coloured printed top a...
16878  23323  tops w women red kurta apparel topwear ethnic ...
12666  14584  tops united colors benetton women solid red to...
15044  43635  tops vero moda women coral coloured top appare...
4867   40667  tops lee women red top apparel topwear casual ...
15690  59796  tops united colors benetton women multi colour...
5096   57898  tops femella women multi coloured top apparel ...
15927  32401  tops arrow woman printed multi coloured top ap...


In [ ]:
pip install FastAPI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.3 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI, Query

In [ ]:
app = FastAPI()

In [ ]:
# Define GET endpoint
@app.get("/recommend")
def get_recommendations(q: str = Query(..., description="Search Keyword"), k: int = 10):
    results = search_products(q, top_k=k)
    return {"query": q, "results": results}

# **adding ratings column**

In [ ]:
df

,id,product
0,15970,shirts turtle check men navy blue shirt appare...
1,39386,jeans peter england men party blue jeans appar...
2,21379,track pants manchester united men solid black ...
3,53759,tshirts puma men grey tshirt apparel topwear c...
4,1855,tshirts inkfruit mens chain reaction tshirt ap...
...,...,...
19572,30614,tshirts nike men striped black jersey apparel ...
19573,13496,tshirts chimp men teja main hoon blue tshirts ...
19574,12544,tshirts tantra women printed peach tshirt appa...
19575,42234,tops sepia women blue printed top apparel topw...


In [ ]:
df['no of ratings'] = np.random.randint(5, 500, size=len(df))
df

,id,product,no of ratings
0,15970,shirts turtle check men navy blue shirt appare...,37
1,39386,jeans peter england men party blue jeans appar...,132
2,21379,track pants manchester united men solid black ...,112
3,53759,tshirts puma men grey tshirt apparel topwear c...,453
4,1855,tshirts inkfruit mens chain reaction tshirt ap...,164
...,...,...,...
19572,30614,tshirts nike men striped black jersey apparel ...,157
19573,13496,tshirts chimp men teja main hoon blue tshirts ...,332
19574,12544,tshirts tantra women printed peach tshirt appa...,282
19575,42234,tops sepia women blue printed top apparel topw...,96


In [ ]:
df ['avg rating'] = np.round(np.random.uniform(2.0, 5.0, size=len(df)),1)
df

,id,product,no of ratings,avg rating
0,15970,shirts turtle check men navy blue shirt appare...,37,2.3
1,39386,jeans peter england men party blue jeans appar...,132,4.7
2,21379,track pants manchester united men solid black ...,112,3.3
3,53759,tshirts puma men grey tshirt apparel topwear c...,453,2.1
4,1855,tshirts inkfruit mens chain reaction tshirt ap...,164,4.6
...,...,...,...,...
19572,30614,tshirts nike men striped black jersey apparel ...,157,4.6
19573,13496,tshirts chimp men teja main hoon blue tshirts ...,332,4.8
19574,12544,tshirts tantra women printed peach tshirt appa...,282,3.0
19575,42234,tops sepia women blue printed top apparel topw...,96,2.8


In [ ]:
df ['price'] = np.random.randint(150, 3000, size=len(df))
df

,id,product,no of ratings,avg rating,price
0,15970,shirts turtle check men navy blue shirt appare...,37,2.3,297
1,39386,jeans peter england men party blue jeans appar...,132,4.7,2855
2,21379,track pants manchester united men solid black ...,112,3.3,1682
3,53759,tshirts puma men grey tshirt apparel topwear c...,453,2.1,1127
4,1855,tshirts inkfruit mens chain reaction tshirt ap...,164,4.6,1668
...,...,...,...,...,...
19572,30614,tshirts nike men striped black jersey apparel ...,157,4.6,420
19573,13496,tshirts chimp men teja main hoon blue tshirts ...,332,4.8,1423
19574,12544,tshirts tantra women printed peach tshirt appa...,282,3.0,1243
19575,42234,tops sepia women blue printed top apparel topw...,96,2.8,1585


In [ ]:
df.to_csv('price_dataset.csv')

# **LIGTHGBM MODEL**

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('price_dataset.csv')

In [ ]:
# TF-IDF Vectorizer on product
vectorizer = TfidfVectorizer(max_features=100)
tfidf_matrix = vectorizer.fit_transform(df['product']).toarray()

In [ ]:
# Add TF-IDF features to DataFrame
for i in range(tfidf_matrix.shape[1]):
    df[f'tfidf_{i}'] = tfidf_matrix[:, i]

# Define features and labels (simulate label = 1 for all for simple relevance)
features = [f'tfidf_{i}' for i in range(tfidf_matrix.shape[1])]
df['label'] = 1  # Dummy relevance since no real click data

<ipython-input-27-674b00319836>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'tfidf_{i}'] = tfidf_matrix[:, i]
<ipython-input-27-674b00319836>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'tfidf_{i}'] = tfidf_matrix[:, i]
<ipython-input-27-674b00319836>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe 

In [ ]:
# Since we don't have real queries, treat everything as a single group
#which is not possible because the lgb can only rank 10,000 data at a time
#so we are grouping the data.
group_size = 500
num_groups = int(np.ceil(len(df) / group_size))
groups = [group_size] * num_groups
groups[-1] = len(df) - (num_groups - 1) * group_size

train_data = lgb.Dataset(df[features], label=df['label'], group=groups)

ranker = lgb.train({
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'verbosity': -1
}, train_data, num_boost_round=50)


In [ ]:
def get_product_ids_for_query(query, top_k=5):
    query_vec = vectorizer.transform([query]).toarray()[0]

    # Duplicate query vector to match shape of product list
    query_df = df.copy()
    for i in range(len(query_vec)):
        query_df[f'tfidf_{i}'] = query_vec[i]

    preds = ranker.predict(query_df[features])
    query_df['score'] = preds
    top_ids = query_df.sort_values(by='score', ascending=False)['id'].head(top_k).tolist()
    return top_ids


In [ ]:
query = "floral red dress"
top_product_ids = get_product_ids_for_query(query)
print(top_product_ids)

[18842, 15970, 39386, 21379, 53759]


<ipython-input-38-394f2ac1c4e9>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_df['score'] = preds


# **Cosine similarity and ranking**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [ ]:
# Load data
df = pd.read_csv("price_dataset.csv")

In [ ]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['product'])


In [ ]:
# Ranking function
def rank_products(query=None, top_k=10):
    if query:
        # Vectorize query
        query_vec = vectorizer.transform([query])
        similarity = cosine_similarity(query_vec, tfidf_matrix).flatten()

        # Get top matching products
        df['similarity'] = similarity
        filtered = df[df['similarity'] > 0]

        if filtered.empty:
            return df.sort_values(by=['avg rating', 'no of ratings'], ascending=False).head(top_k)

        # Rank based on avg_rating and price (you can fine-tune the weightage)
        filtered['score'] = (
            filtered['similarity'] * 0.5 +
            filtered['avg rating'] / 5 * 0.3 +      # Normalize rating to [0,1]
            1 - (filtered['price'] / filtered['price'].max()) * 0.2  # Prefer lower prices
        )

        return filtered.sort_values(by='score', ascending=False).head(top_k)[['id', 'product']]
    else:
        # No query: return top 10 rated
        return df.sort_values(by=['avg rating', 'no of ratings'], ascending=False).head(top_k)[['id', 'product']]


In [ ]:
# Example usage
print(rank_products("floral dress"))     # Search with keyword

          id                                            product
6238    8484  dresses forever new women floral purple dress ...
16911  41565  dresses women blue dress apparel dress casual ...
5457   59795  dresses united colors benetton women blue dres...
17086  57052  dresses elle women blue dress apparel dress ca...
15667  39217  dresses femella women floral red dress apparel...
3236   45777  dresses remanika women red dress apparel dress...
12658  58499  dresses tonga white floral design dress appare...
12741  34578  dresses tonga women printed red dress apparel ...
16472  57037  dresses elle women blue dress apparel dress ca...
15465  11017  tops wrangler women floral print blue top appa...


<ipython-input-52-7efa790aaa66>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['score'] = (


In [ ]:
print(rank_products())                   # No keyword

          id                                            product
6498   13774  tshirts belmonte men printed black tshirts app...
15517  19626  tshirts united colors benetton men printed whi...
14339  59379  kurtas vishudh women orange printed kurta appa...
367     7532  tshirts nike mens encourage blue tshirt appare...
974     7937  tshirts puma mens tech graphic green black tsh...
10969   1651  tshirts domyos women style pink tshirt apparel...
18387  42667  kurtas alma women blue kurta apparel topwear e...
5987    3292  tshirts guerrilla mens fighter blue tshirt app...
9139   57988  dresses latin quarters women orange dress appa...
14670  16272  shirts levis men red white striped shirt appar...


# **Preparing data for DB**

In [ ]:
db = pd.read_csv('images.csv')
db

,filename,link,masterCategory,subCategory
0,15970.jpg,http://assets.myntassets.com/v1/images/style/p...,Apparel,Topwear
1,39386.jpg,http://assets.myntassets.com/v1/images/style/p...,Apparel,Bottomwear
2,59263.jpg,http://assets.myntassets.com/v1/images/style/p...,Accessories,Watches
3,21379.jpg,http://assets.myntassets.com/v1/images/style/p...,Apparel,Bottomwear
4,53759.jpg,http://assets.myntassets.com/v1/images/style/p...,Apparel,Topwear
...,...,...,...,...
44441,17036.jpg,http://assets.myntassets.com/v1/images/style/p...,Footwear,Shoes
44442,6461.jpg,http://assets.myntassets.com/v1/images/style/p...,Footwear,Flip Flops
44443,18842.jpg,http://assets.myntassets.com/v1/images/style/p...,Apparel,Topwear
44444,46694.jpg,http://assets.myntassets.com/v1/images/style/p...,Personal Care,Fragrance


In [ ]:
df = pd.read_csv('style.csv')
df

,id,gender,mcat,scat,name,color,season,usage,brand,price,no of ratings,avg rating
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,Casual,Turtle Check Men Navy Blue Shirt,297,37,2.3
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,Casual,Peter England Men Party Blue Jeans,2855,132,4.7
2,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,Casual,Manchester United Men Solid Black Track Pants,1682,112,3.3
3,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,Casual,Puma Men Grey T-shirt,1127,453,2.1
4,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,Casual,Inkfruit Mens Chain Reaction T-shirt,1668,164,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...
19572,30614,Men,Apparel,Topwear,Tshirts,Black,Summer,Sports,Nike Men Striped Black Jersey,420,157,4.6
19573,13496,Men,Apparel,Topwear,Tshirts,Blue,Fall,Casual,Chimp Men Teja Main Hoon Blue Tshirts,1423,332,4.8
19574,12544,Women,Apparel,Topwear,Tshirts,Peach,Fall,Casual,Tantra Women Printed Peach T-shirt,1243,282,3.0
19575,42234,Women,Apparel,Topwear,Tops,Blue,Summer,Casual,Sepia Women Blue Printed Top,1585,96,2.8


In [ ]:
import pandas as pd

# Load both CSVs
images_df = pd.read_csv('images.csv')  # should have 'filename' and 'link'
style_df = pd.read_csv('style.csv')    # should have 'id'

# Remove ".jpg" from filename to match 'id'
images_df['id'] = images_df['filename'].str.replace('.jpg', '', regex=False)

# Ensure ID types match (convert both to int or str)
images_df['id'] = images_df['id'].astype(int)
style_df['id'] = style_df['id'].astype(int)

# Merge based on 'id'
merged_df = style_df.merge(images_df[['id', 'link']], on='id', how='left')

# Rename column
merged_df = merged_df.rename(columns={'link': 'url'})

# Optional: Save the result
merged_df.to_csv('final_output.csv', index=False)

print(merged_df)


          id gender     mcat        scat         name      color  season  \
0      15970    Men  Apparel     Topwear       Shirts  Navy Blue    Fall   
1      39386    Men  Apparel  Bottomwear        Jeans       Blue  Summer   
2      21379    Men  Apparel  Bottomwear  Track Pants      Black    Fall   
3      53759    Men  Apparel     Topwear      Tshirts       Grey  Summer   
4       1855    Men  Apparel     Topwear      Tshirts       Grey  Summer   
...      ...    ...      ...         ...          ...        ...     ...   
19572  30614    Men  Apparel     Topwear      Tshirts      Black  Summer   
19573  13496    Men  Apparel     Topwear      Tshirts       Blue    Fall   
19574  12544  Women  Apparel     Topwear      Tshirts      Peach    Fall   
19575  42234  Women  Apparel     Topwear         Tops       Blue  Summer   
19576  18842    Men  Apparel     Topwear      Tshirts       Blue    Fall   

        usage                                          brand  price  \
0      Casual   